# KNN User-User Collaborative Filtering

The idea is to find similar users based on their preferences or behavior and recommend items that similar users liked but the target user has not interacted with. If User A and User B have rated a lot of the same items similarly, User B might enjoy items that User A has rated highly but hasn’t yet discovered.

Based on [Introduction to Recommender System](https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada):

> Collaborative methods for recommender systems are methods that are **based solely on the past interactions** recorded between users and items in order to produce new recommendations. These interactions are stored in the so-called “user-item interactions matrix”. Then, the main idea that rules collaborative methods is that these past user-item interactions are sufficient to detect similar users and/or similar items and make predictions based on these estimated proximities.

> User-User CF is **applying KNN to find the neighbor users** with similar prior item preferences, and prediction of target item I for user U will be computed from the preferences of the neighbors.
Notice that, when computing similarity between users, the number of “common interactions” (how much items have already been considered by both users?) should be considered carefully! Indeed, most of the time, we want to avoid that someone that only have one interaction in common with our reference user could have a 100% match and be considered as being “closer” than someone having 100 common interactions and agreeing “only” on 98% of them. So, we consider that two users are similar if they have interacted with a lot of common items in the same way (similar rating, similar time hovering…).

In this notebook, we would take a very simple and straight-forward approach to implement KNN User-User CF. We formulate the problem as predicting the rating between user U and item I based on the rating records.

Specifically, we will:
- Collect input containing user-item rating
- Build user-user similarity matrix
- Measure similarities between users by using plain item rating vectors
- Predict the rating between user U and item I by getting all users who have rated I, identify N neighbors and weighted average their ratings based on how similar the users are to the user U

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.math_utils import sigmoid
from src.model import User2UserCollaborativeFiltering
from src.train_utils import map_indice
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L4 - Reco Algo"
    run_name: str = "002-cf-u2u"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-10-18 16:52:09.603 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L4 - Reco Algo - run 002-cf-u2u...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L4 - Reco Algo",
  "run_name": "002-cf-u2u",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/002-cf-u2u",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(n_users, n_items):
    model = User2UserCollaborativeFiltering(n_users, n_items)
    return model

# Test implementation

In [5]:
# Mock data
user_indices = [0, 0, 1, 1, 2, 2, 2]
item_indices = [0, 1, 1, 2, 3, 1, 2]
ratings = [1, 4, 4, 5, 3, 2, 4]
n_users = len(set(user_indices))
n_items = len(set(item_indices))

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]

print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)

model = init_model(n_users, n_items)

users = [0, 1, 2]
items = [2, 2, 0]
predictions = model.predict(users, items)
print(predictions)

Mock User IDs: [0, 0, 1, 1, 2, 2, 2]
Mock Item IDs: [0, 1, 1, 2, 3, 1, 2]
Ratings: [1, 4, 4, 5, 3, 2, 4]
[0.5 0.5 0.5]


In [6]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.99031217 0.98201379 0.73105858]


#### 🧐 Go into details

In [7]:
model.user_item_matrix

array([[1., 4., 0., 0.],
       [0., 4., 5., 0.],
       [0., 2., 4., 3.]])

In [8]:
model.user_similarity

array([[0.        , 0.60604322, 0.36030188],
       [0.60604322, 0.        , 0.81202071],
       [0.36030188, 0.81202071, 0.        ]])

In [9]:
# Inspect one example
user = 0
item = 2

sim_scores = model.user_similarity[user]
print(f"{sim_scores=}")

sim_scores=array([0.        , 0.60604322, 0.36030188])


In [10]:
# Only consider users who have rated the item
user_ratings = model.user_item_matrix[:, item]
print(f"{user_ratings=}")
sim_scores = sim_scores[user_ratings != 0]
print(f"{sim_scores=}")
user_ratings = user_ratings[user_ratings != 0]
print(f"{user_ratings=}")

user_ratings=array([0., 5., 4.])
sim_scores=array([0.60604322, 0.36030188])
user_ratings=array([5., 4.])


In [11]:
# Weighted average of ratings
print(f"Weighted average: {np.dot(sim_scores, user_ratings)}")
print(f"Normalization factor: {np.sum(sim_scores)}")
print(f"Predicted rating: {np.dot(sim_scores, user_ratings) / np.sum(sim_scores)}")
print(
    f"Predicted rating - sigmoid: {sigmoid(np.dot(sim_scores, user_ratings) / np.sum(sim_scores))}"
)

Weighted average: 4.471423593469625
Normalization factor: 0.9663450945516922
Predicted rating: 4.627149885356445
Predicted rating - sigmoid: 0.9903121712214553


In [12]:
recommendations = model.recommend(val_user_indices, k=2)

Generating recommendations:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
recommendations

{'user_indice': [0, 0, 1, 1, 2, 2],
 'recommendation': [2, 1, 2, 3, 2, 1],
 'score': [0.9903121712214553,
  0.9628273118576526,
  0.9820137900379085,
  0.9525741268224334,
  0.9933071490757153,
  0.9820137900379085]}

# Prep data

In [14]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=1628641464793


In [15]:
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))
n_users = len(unique_user_ids)
n_items = len(unique_item_ids)

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-10-18 16:52:10.933 | INFO     | __main__:<module>:8 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [16]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)

user_indices = [idm.get_user_index(user_id) for user_id in user_ids]
item_indices = [idm.get_item_index(item_id) for item_id in item_ids]
ratings = train_df[args.rating_col].values.tolist()

val_user_indices = [idm.get_user_index(user_id) for user_id in val_df[args.user_col]]
val_item_indices = [idm.get_item_index(item_id) for item_id in val_df[args.item_col]]
val_ratings = val_df[args.rating_col].values.tolist()

# Train

In [17]:
model = init_model(n_users, n_items)

#### Predict before train

In [18]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence
652,AE5QGKFJBTNUNXPIAFEFTAAH3XRA,B00YM7AKLG,0.0,1630436382845,12717,3793,Video Games,FIFA 16 - Standard Edition - PlayStation 4,"[*The DLC (Downloadable Content), Trials/Subsc...","[Video Games, PlayStation 4]",25.98,"[-1, -1, -1, -1, 3774, 4030, 4259, 685, 1722, ..."
1170,AE5QGKFJBTNUNXPIAFEFTAAH3XRA,B07L3SXCL9,1.0,1630436382845,12717,4359,All Electronics,Jeecoo V20 Stereo Gaming Headset for PS4 PS5 X...,[],"[Video Games, PC, Accessories, Headsets]",29.99,"[-1, -1, -1, -1, 3774, 4030, 4259, 685, 1722, ..."


In [19]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])

2024-10-18 16:52:11.180 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AE5QGKFJBTNUNXPIAFEFTAAH3XRA and parent_asin = B07L3SXCL9


array([0.5])

#### Training loop

In [20]:
model.fit(user_indices, item_indices, ratings)

# Predict

In [21]:
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.predict([user_indice], [item_indice])

2024-10-18 16:52:23.541 | INFO     | __main__:<module>:1 - Test predicting before training with user_id = AE5QGKFJBTNUNXPIAFEFTAAH3XRA and parent_asin = B07L3SXCL9


array([0.5])

# Evaluate

## Ranking metrics

In [22]:
# This can take about 10 mins -> TODO: Optimize
recommendations = model.recommend(val_user_indices, k=args.top_K)

Generating recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

In [23]:
recommendations_df = pd.DataFrame(recommendations).pipe(create_rec_df, idm)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,7087,0,0.993307,1.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B003R50910
1,7087,2744,0.993307,2.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B079MLPDRJ
2,7087,3791,0.993307,3.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B001EYUTMA
3,7087,1307,0.993307,4.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B00936K09S
4,7087,3793,0.993307,5.0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B00YM7AKLG
...,...,...,...,...,...,...
189795,14319,3068,0.993307,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B000FUIS40
189796,14319,580,0.993307,197.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B001G605ZC
189797,14319,584,0.993307,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B01GDVGQ48
189798,14319,586,0.993307,199.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B087STK7JZ


In [24]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
1757,AGDJVPSADB7OTS2CL4TLYQM4CT4A,B087NNPYP3,5.0,1.0
145,AFJKZEYD2VZSI2NO3JZNMA4XX4RA,B09MRM36JJ,4.0,1.0
170,AFBRTNVOROW7UVA66UPX5YCFC6MQ,B00EP2WNKY,3.0,1.0
1729,AGENEGNVXK2GM4YG35LAYXBO527Q,B071Y568B7,4.0,1.0
87,AEL7SJG67X3FAH6U4KLHBOTF6PZA,B001EYUWDG,5.0,1.0
...,...,...,...,...
393,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0051D8PGM,0.0,18.0
1138,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B00800VJWU,0.0,18.0
1845,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B0050SVMYA,0.0,18.0
1677,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0088TN73M,0.0,19.0


In [25]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
50,1416.0,4695.0,0.993307,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B003MZ23LY,0,NaN
181,1416.0,2495.0,0.993307,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B07YNJBDK3,0,NaN
74,1416.0,2448.0,0.993307,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B0050SY8UK,0,NaN
118,1416.0,2450.0,0.993307,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B00VMB5VCS,0,NaN
36,1416.0,2455.0,0.993307,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B001EYUQ2I,0,NaN
...,...,...,...,...,...,...,...,...
191630,17528.0,1030.0,0.993307,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B081TQRCLM,0,NaN
191474,17528.0,2992.0,0.993307,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B001EYUV3M,0,NaN
191498,17528.0,1020.0,0.993307,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B0050SVNSU,0,NaN
191538,17528.0,3015.0,0.993307,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00BGAA0SU,0,NaN


In [26]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [27]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values

In [28]:
classifications = model.predict(val_user_indices, val_item_indices)

In [29]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,classification_proba,label
0,AEWCUX5UKUYPDZJIOB6XMLCBJ3KA,B0BLFYF8K2,4.0,1630263342566,7087,3330,Computers,"Logitech G600 MMO Gaming Mouse, RGB Backlit, 2...","[With 20 buttons, the Logitech G600 MMO Gaming...","[Video Games, PC, Accessories, Gaming Mice]",37.99,"[1596, 1047, 3310, 2885, 1862, 1299, 4500, 209...",0.981448,1
1,AFFPVZ3JNCTQIKAK4XK37E2ENWWA,B00HVBPRUO,4.0,1655428133046,4199,3721,Video Games,Gold Wireless Stereo Headset - PlayStation 4,[A Headset for Gamers: Experience everything f...,"[Video Games, PlayStation 4, Accessories, Head...",None,"[-1, -1, 3810, 3019, 1158, 1714, 98, 309, 2291...",0.989572,1
2,AHAIICWIZT6PYSS5QJNFYP6ZXLCA,B001EYUPJW,0.0,1628811542081,18270,3274,Video Games,Def Jam Fight for NY - Gamecube (Gold),"[The ultimate hip-hop fueled fighting game, De...","[Video Games, Legacy Systems, Nintendo Systems...",349.97,"[2245, 705, 3089, 4462, 3922, 2808, 3476, 3558...",0.880797,0
3,AF2AAA4CWRVF2IYVE7WB6OOIEMFA,B072C3VM5F,0.0,1635286957988,10410,4179,Video Games,Far Cry 5 Gold Edition - Xbox One [Digital Code],[],"[Video Games, Xbox One, Games]",None,"[-1, -1, -1, -1, 1690, 431, 720, 742, 1885, 2711]",0.993307,0
4,AFBRTNVOROW7UVA66UPX5YCFC6MQ,B07YBXFDYK,3.0,1636189764550,9102,2986,Video Games,The Evil Within 2 - PlayStation 4,"[From Shinji Mikami, The Evil Within 2 takes t...","[Video Games, PlayStation 4, Games]",20.98,"[-1, -1, -1, -1, 1853, 195, 4616, 1777, 4654, ...",0.992385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AGNZODFXG6WNNJUAKR3MI42SDG5A,B0B8RG61GK,0.0,1642617264556,10449,3436,Computers,Fast Charging Cable for Switch/Switch Lite/Swi...,"[Dimensions:, Length: 9.8 Feet/3 Meters]","[Video Games, Legacy Systems, Nintendo Systems...",9.89,"[-1, -1, -1, -1, 2074, 1310, 3877, 184, 2952, ...",0.500000,0
1894,AHA6LZWVG2U4WBXNZRWCESNJXNUA,B002JTX87C,0.0,1646645847748,16655,380,Video Games,Scooby Doo! First Frights NDS,"[Product Description, In Scooby-Doo! First Fri...","[Video Games, Legacy Systems, Nintendo Systems...",43.77,"[1208, 1088, 3365, 4287, 3679, 4048, 2351, 197...",0.988464,0
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,11656,4503,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 2159, 3248, 48, 445, 4343, 584, 907, 4155...",0.731059,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,1562,4543,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 4507, 1351, 132, 4660, 1979]",0.980681,1


In [30]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# Clean up

In [31]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/10/18 17:00:53 INFO mlflow.tracking._tracking_service.client: 🏃 View run 002-cf-u2u at: http://localhost:5003/#/experiments/1/runs/21a9d4d864744c7b96db4fe276a909b1.
2024/10/18 17:00:53 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.


# Appendix

## Model returning same score for every user-item in top 100